# Plots and figures Rockland

This notebook contains all code to generate the figures in the associated publications.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import os.path as path
import os
import socket
import sys

In [ ]:
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
print("Your style sheets are located at: {}".format(path.join(mpl.__path__[0], 'mpl-data', 'stylelib')))

import numpy as np
import pandas as pd
import ptitprince as pt
import seaborn as sns

%matplotlib inline

In [ ]:
sys.path.append('..')

from configs.configs import get_config_dict
from helpers.figures import set_size

## Set configurations (shared across all figures)

In [ ]:
data_split = 'all'
kernel_param = 'kernel_lengthscales'
model_name = 'SVWP_joint'
pp_pipeline = 'custom_fsl_pipeline'

In [ ]:
rockland_cfg = get_config_dict(
    data_set_name='rockland',
    subset='645',
    hostname=socket.gethostname()
)

## Time series

### Fig S13A. - Time series

In [ ]:
from datasets.fmri.tb.rockland.plotters.plot_node_timeseries import _plot_node_timeseries_average_over_subjects, _plot_node_timeseries
from helpers.data import normalize_array
from helpers.rockland import get_rockland_subjects, get_convolved_stim_array

In [ ]:
roi_list_name = 'final'

# Get stimulus time series (convolved with HRF) and normalize for better interpretation.
convolved_stim_array = get_convolved_stim_array(config_dict=rockland_cfg)
convolved_stim_array = normalize_array(convolved_stim_array, verbose=False)

all_subjects_list = get_rockland_subjects(config_dict=rockland_cfg)

for i_subject, subject in enumerate(all_subjects_list):
    # print(f'\n> Subject {i_subject+1:d} / {len(all_subjects_list):d}: {subject:s}\n')
    subject_timeseries_df = pd.read_csv(
        os.path.join(rockland_cfg['data-basedir'], pp_pipeline, 'node_timeseries', roi_list_name, subject)
    )  # (N, D)
    if i_subject == 0:
        # print(subject_timeseries_df)
        mean_over_subjects_timeseries_df = subject_timeseries_df.copy()
    else:
        mean_over_subjects_timeseries_df += subject_timeseries_df.values
mean_over_subjects_timeseries_df /= len(all_subjects_list)
# print(mean_over_subjects_timeseries_df)

_plot_node_timeseries_average_over_subjects(
    config_dict=rockland_cfg,
    ts_df=mean_over_subjects_timeseries_df,
    convolved_stim_array=convolved_stim_array,
    # figures_savedir="/Users/onnokampman/Developer/ABL"
)

## TVFC estimates

### Fig. 11A - TVFC estimates (run on Hivemind)

In [ ]:
from benchmarks.fmri.tb.rockland.plotters.plot_TVFC_estimates import plot_average_over_subject_tvfc_estimates_joint
from helpers.rockland import get_rockland_subjects

In [ ]:
all_subjects_list = get_rockland_subjects(config_dict=rockland_cfg)


plot_average_over_subject_tvfc_estimates_joint(
    config_dict=rockland_cfg,
    data_split=data_split,
    preprocessing_pipeline=pp_pipeline,
    all_subjects_list=all_subjects_list,
    edges_to_plot_indices=rockland_cfg['roi-edges-list'],
    column_names=rockland_cfg['roi-list'],
    # figures_savedir="/Users/onnokampman/Developer",
)

## GLM beta

### Fig. 11B - GLM betas

In [ ]:
from benchmarks.fmri.tb.rockland.prediction_benchmark.plot_glm_beta import plot_glm_beta_bar, _clean_up_model_names, _clean_up_regressor_names
from helpers.rockland import get_edges_names

In [ ]:
all_glm_betas_df = pd.DataFrame(
    columns=get_edges_names(rockland_cfg)
)
for model_name in rockland_cfg['stimulus-prediction-models']:
    betas_savedir = os.path.join(rockland_cfg['git-results-basedir'], 'prediction_benchmark')
    betas_df = pd.read_csv(
        os.path.join(betas_savedir, f'betas_df_{model_name:s}.csv'),
        index_col=0
    )  # (n_edges, n_regressors)
    betas_df = _clean_up_regressor_names(betas_df)
    all_glm_betas_df.loc[model_name, :] = betas_df['stim']

all_glm_betas_df = all_glm_betas_df.loc[rockland_cfg['plot-stimulus-prediction-models'], :]
all_glm_betas_df = _clean_up_model_names(all_glm_betas_df)


plot_glm_beta_bar(
    config_dict=rockland_cfg,
    stimulus_betas_df=all_glm_betas_df,
    figures_savedir="/Users/onnokampman/Developer/"
)

## Imputation benchmark

### Fig. 15 - Imputation benchmark

In [ ]:
from benchmarks.fmri.tb.rockland.imputation_study.plot_LEOO_likelihoods import plot_likelihoods_raincloud

In [ ]:
data_split = 'LEOO'

all_test_likelihoods_df = pd.DataFrame()
for model_name in rockland_cfg['plot-likelihoods-models']:
    likelihoods_filename = f'{data_split:s}_likelihoods_{model_name:s}.csv'
    test_likelihoods_savedir = os.path.join(rockland_cfg['git-results-basedir'], 'imputation_study')
    likelihoods_df = pd.read_csv(
        os.path.join(test_likelihoods_savedir, likelihoods_filename),
        index_col=0
    )  # (n_subjects, 1)
    likelihoods_array = likelihoods_df.values.flatten()  # (n_subjects, )

    model_name = model_name.replace('SVWP_joint', 'WP')
    model_name = model_name.replace('_joint', '-J')
    model_name = model_name.replace('DCC_bivariate_loop', 'DCC-BL')
    model_name = model_name.replace('SW_cross_validated', 'SW-CV')
    model_name = model_name.replace('_', '-')

    all_test_likelihoods_df[model_name] = likelihoods_array


plot_likelihoods_raincloud(
    config_dict=rockland_cfg,
    all_test_likelihoods_df=all_test_likelihoods_df,
    # figures_savedir="/Users/onnokampman/Developer/"
)

## Kernel lengthscales and optimal window length

In [ ]:
from helpers.kernel_analysis import plot_lengthscale_window_length_relation

In [ ]:
kernel_params_savedir = os.path.join(
    rockland_cfg['git-results-basedir'], 'kernel_analysis', data_split, model_name
)
kernel_params_df = pd.read_csv(
    os.path.join(kernel_params_savedir, f'{kernel_param:s}_kernel_params.csv'),
    index_col=0
)
print(kernel_params_df)
optimal_window_lengths_df = pd.read_csv(
    os.path.join(
        rockland_cfg['git-results-basedir'], 'optimal_window_lengths', data_split, 'optimal_window_lengths.csv'
    ),
    index_col=0
)
print(optimal_window_lengths_df)

# Prepare data for plot.
assert kernel_params_df.shape == optimal_window_lengths_df.shape
kernel_params_array = kernel_params_df.values.reshape(-1, 1)
optimal_window_lengths_array = optimal_window_lengths_df.values.reshape(-1, 1)

# Remove outliers. TODO: must be a better way of doing this
kernel_params_array[kernel_params_array > 0.4] = 0.4

In [ ]:
plot_lengthscale_window_length_relation(
    rockland_cfg,
    kernel_params_array,
    optimal_window_lengths_array,
    # figures_savedir="/Users/onnokampman/Developer/"
)